In [ ]:
import numpy as np
from numba import jit

import matplotlib.pyplot as plt

from functools import partial

Instead of copy-pasting every routine in miniMD, the cell below contains a quick hack to import miniMD modules directly from the folder 

In [ ]:
import sys
sys.path.append('../miniMD')
from miniMD import *

# Free Energy Differences

### Definition of Source and Target Systems

First of all we define a "source" and a "target" system, setting energy and force functions for both of them. These represent the two "situations" between which we want to calculate the free energy difference. In many cases we use as a source system the ideal gas $U_{id} = 0$ and as target the system we are interested in.

In this case we use as source a 2d-harmonic oscillator (we have seen the 1d case in the previous exercise) and as target a 2d double-well.
Given the current configuration $x = (x_0,x_1)$, the two systems have potential energies:
$$
U_{HO} = \frac{1}{2} k (x - r)^2
$$
$$
U_{DW} = B\bigl[(x_0^2 - 1)^2 + (x_0 - x_1)^2\bigr]
$$
where $k$, $r$ are the force constant and rest position of the harmonic oscillator (HO), respectively and $B$ is a scaling factor for the double well model (DW).

In [ ]:
@jit(nopython=True) 
def energy_function_source(current_x : np.ndarray) -> float:
    """
    Calculates the potential energy of the source system given a 
    configuration current_x. The example here is for a 2d 
    harmonic oscillator with a force constant of 10 and rest 
    position of (0, 0).

    Parameters
    ----------
    current_x : np.ndarray
        Current 2d configuration.

    Returns
    -------
    U : float
        Potential energy of the given configuration

    """

    rest_position = np.zeros(2)
    force_constant = 10

    return 1/2 * force_constant * ((current_x[0] - rest_position[0])**2 + (current_x[1] - rest_position[1])**2)

@jit(nopython=True) 
def force_function_source(current_x : np.ndarray) -> np.ndarray:
    """
    Calculates the force of the source system given a 
    configuration current_x. The example here is for a 
    2d harmonic oscillator with a force constant of 10 and rest 
    position of (0, 0).

    Remember: The force is the negative gradient of the potential energy with 
    respect to the current configuration. Therefore, it must be of the same 
    dimensionality as the configuration.

    Parameters
    ----------
    current_x : np.ndarray
        Current configuration.

    Returns
    -------
    force : np.ndarray
        Force corresponding the given configuration.

    """

    rest_position = np.zeros(2)
    force_constant = 10

    force = np.zeros(2)

    force[0] = -force_constant * (current_x[0] - rest_position[0])
    force[1] = -force_constant * (current_x[1]- rest_position[1])

    return force


@jit(nopython=True) 
def energy_function_target(current_x : np.ndarray) -> float:
    """
    Calculates the potential energy of the target system given a 
    configuration current_x. The example here is for a 2d 
    double well with a scaling constant of 3.

    Parameters
    ----------
    current_x : np.ndarray
        Current 2d configuration.

    Returns
    -------
    U : float
        Potential energy of the given configuration

    """
    
    B = 3

    return B * ((current_x[0]**2 - 1)**2 + (current_x[0] - current_x[1])**2)

@jit(nopython=True) 
def force_function_target(current_x : np.ndarray) -> np.ndarray:
    """
    Calculates the force of the target system given a 
    configuration current_x. The example here is for a 
    2d double well with scaling constant of 3.

    Parameters
    ----------
    current_x : np.ndarray
        Current configuration.

    Returns
    -------
    force : np.ndarray
        Force corresponding the given configuration.

    """
    B = 3

    force = np.zeros(2)
    
    force[0] = -2 * B * (2 * current_x[0]**3 -current_x[0] - current_x[1])
    force[1] =  2 * B * (current_x[0] - current_x[1])

    return force

A plot of the source and target energy landscapes is shown below

In [ ]:
x_values = np.linspace(-3, 3, 100)
y_values = np.linspace(-3, 3, 100)

x_grid, y_grid = np.meshgrid(x_values, y_values)
source_energy = np.zeros((len(x_values), len(y_values)))
target_energy = np.zeros((len(x_values), len(y_values)))

fig, ax = plt.subplots(1, 2, figsize=(10,4), dpi=180)

for i in range(len(x_values)):
    for j in range(len(y_values)):

        source_energy[i,j] = energy_function_source(np.array([x_grid[i, j], y_grid[i, j]]))
        target_energy[i,j] = energy_function_target(np.array([x_grid[i, j], y_grid[i, j]]))

ax[0].set_xlabel(r"$x_0$")
ax[0].set_ylabel(r"$x_1$")
ax[0].set_title(r"Source")
image = ax[0].contourf(x_grid, y_grid, source_energy, levels=np.linspace(0, 20, 20))
plt.colorbar(image)

ax[1].set_xlabel(r"$x_0$")
ax[1].set_ylabel(r"$x_1$")
ax[1].set_title(r"Target")
image = ax[1].contourf(x_grid, y_grid, target_energy, levels=np.linspace(0, 20, 20))

plt.colorbar(image)
plt.show()

## Thermodynamic Integration

Thermodynamic integration consists in varying the potential energy function of the system along a certain protocol and to compute free energy difference as an integrals along this protocol. This means to define a certain $U(x; \lambda)$ where $\lambda \in [0,1]$ is the protocol control parameter, such that, in our case, $U(x; \lambda=0) = U_{HO}$ and $U(x; \lambda=1) = U_{DW}$. With the exception of the boundary conditions on $\lambda$ he chosen protocol is arbitrary and the simplest choice is
$$
U(x; \lambda) = (1-\lambda)U_{HO} + \lambda U_{DW}
$$
which clearly satisfies the required boundary conditions on $\lambda$. The free energy difference is then computed as
$$
\Delta F = \int_0^1 \text{d}\lambda \Braket{\frac{\text{d}U}{\text{d}\lambda}}_\lambda
$$

As a first step, we define a general method that accepts current configuration $x$, the current value of the protocol control parrameter $\lambda$ and two **functions** (`callable` in python) $f$ and $g$. The method returns the **value** of 
$$
h(x; \lambda) = (1-\lambda)f(x) + \lambda g(x)
$$

Moreover, given the protocol, we also want to have a method that computes its derivative. As above, given $x$, $\lambda$, $f$ and $g$, the method should return the **value** of 
$$
\frac{\text{d}h(x)}{\text{d}\lambda} = g(x) - f(x)
$$ 

In [ ]:
def linear_switching_protocol(current_x : np.ndarray, llambda : float, f : callable, g : callable) -> np.ndarray:

    return ...

def derivative_linear_switching_protocol(current_x : np.ndarray, llambda : float, f : callable, g : callable) -> np.ndarray:

    return ...

The thermodynamic integration algorithm to compute free energy difference can then be summarized as follows:
1. Carry out $L$ different simulations for different values of the parameter $\lambda_i$ covering the interval $[0,1]$
2. In each siomulation compute $\Braket{\frac{\text{d}U}{\text{d}\lambda}}_{\lambda_i}$
3. Compute numerically the integral for $\Delta F$.

As a first step we choose to do every simulation equally long and to perform simulations for equally spaced values of $\lambda$.

In [ ]:
l_simulations = 9
lambdas = np.linspace(0, 1, l_simulations)

equilibration_steps = 10000
n_steps_per_simulation = 1000000
output_frequency = 10

beta = 1
timestep = 0.001
diffusion_coefficient = 1

initial_x = np.zeros(2)

assert equilibration_steps < n_steps_per_simulation, "Make sure you don't equilibrate longer than you simulate."
assert output_frequency < n_steps_per_simulation, "Make sure you don't output less often than you simulate."
assert output_frequency > 0, "The output frequency needs to be larger than 0"


Our usual MD cycle will be then embedded in a loop over the different $\lambda$, in order to perform one simulation per value $\lambda_i$.

The definition of the force for each simulation can be done via the defined method for the linear switching. Indeed, it is easy to show how the same protocol used to switch between the potential energies of source and tareget systems can be used straight away for forces (convince yourself about this). Therefore, the easiest way is to apply the protocol at each step of the simulation.

A more elegant way is to use the method `partial` from `functools`. This method gets as input a function and some of its input values and return a new function where the input values passed are automatically given. Here is an example:

```
[0] from functools import partial

[1] def foo(x, a, b):

        return a*x + b
        
[2] foo(x=2, a=1, b=3)
[ ] 5

[3] bar = partial(foo, a=1, b=3)

[4] bar(x=2)
[ ] 5

```
This amounts to perform point 1. of the algorithm.

For each simulation it is important to save the computed average $\frac{\text{d}U}{\text{d}\lambda}$. To this end, again two routes are possible: The use of the protocol at each time step or the definition of a partial method for each $\lambda$.

This is point 2.

In [ ]:
simulation_trajectories = []
simulation_avg_dudl = []

for l in lambdas:

    previous_x = initial_x.copy()
    
    total_frames = int(np.ceil((n_steps_per_simulation - equilibration_steps) / output_frequency))
    trajectory = np.zeros((total_frames , 2))
    dudl = np.zeros((total_frames))

    for step in range(n_steps_per_simulation):

        current_force = ...
        
        previous_x = update_positions(previous_x, current_force, beta, timestep, diffusion_coefficient)

        if step >= equilibration_steps and step % output_frequency == 0:
            index = (step - equilibration_steps) // output_frequency
            trajectory[index] = previous_x
            dudl[index] = ...
            
    simulation_trajectories.append(trajectory)
    simulation_avg_dudl.append(dudl.mean())

Finally point 3. needs to be performed, i.e. the numerical integration of the obtained values for the mean derivatives of the energy with respect to the protocol. This can be done with any algorithm for numerical integration, but a very commonly adopted one is the trapezoidal rule, which is available in numpy via `numpy.trapz`. 

In [ ]:
def compute_free_energy_difference(lambdas, dudlambdas):
    
    return np.trapz(dudlambdas, lambdas)

It is now possible to obtain the free energy difference between the source and target systems.<br>
How would you check the result?

In [ ]:
delta_F_ti = compute_free_energy_difference(lambdas, np.array(simulation_avg_dudl))
print(f"∆F = {delta_F_ti}")

Saving the trajectories obtained for each $\lambda_i$ it is also possible to visualize the evolution of the trajectories together with the switching of the potential energy surface. An example is reported below.

In [ ]:
# Generating closest square grid out of l_simulations
nr = int(l_simulations**0.5)
nc = int(np.ceil(l_simulations/float(nr)))

# Approximate grid size based on nc and nr
fig, ax = plt.subplots(nrows=nr, ncols=nc, figsize=(6*nc,6*nr), dpi=180)

# Taking care of degenerate grids
if l_simulations == 1:
    ax = np.array([ax])
if nr == 1:
    ax = ax.reshape(1, nc)
if nc == 1:
    ax = ax.reshape(nr, 1)

# Generating grid for contour plots
x_values = np.linspace(-3, 3, 100)
y_values = np.linspace(-3, 3, 100)

x_grid, y_grid = np.meshgrid(x_values, y_values)
switching_energies = np.zeros((len(x_values), len(y_values)))

# Looping over plot grid 
for il in range(nr):
    for jl in range(nc):
        
        # Finding corresponding simulation
        lindx = il*nc + jl
        if lindx < l_simulations:
            l = lambdas[lindx]
            t = simulation_trajectories[lindx]    
            total_energy_wrapped = partial(linear_switching_protocol, llambda=l, f=energy_function_source, g=energy_function_target)

            # Generating contour plots
            for i in range(len(x_values)):
                for j in range(len(y_values)):

                    switching_energies[i,j] = total_energy_wrapped(np.array([x_grid[i, j], y_grid[i, j]]))

            ax[il][jl].set_title(r"$\lambda = {:.3g}$".format(l))
            ax[il][jl].set_xlabel(r"$x_0$")
            ax[il][jl].set_ylabel(r"$x_1$")
        
            ax[il][jl].contour(x_grid, y_grid, switching_energies, levels=np.linspace(0, 20, 10))
            ax[il][jl].plot(t[:,0], t[:,1], c=f"C0", lw=1)
        else: 
            # Taking care of empty plots
            ax[il][jl].axis("off")

plt.show()

## Free Energy Perturbation

Free Energy perturbation is another method for computing free energies between a state A and a state B. It has been introduced by Zwanzing in 1954. In the standard decomposition of the free-energy difference
$$
\Delta F_{AB} = F_B - F_A = k_BT\log Z_B - k_BT \log Z_A = k_BT\log \frac{Z_B}{Z_A} 
$$
the ratio between the partition functions can be expressed as
$$
\frac{Z_B}{Z_A} = \frac{\int \text{d}x e^{-\beta U_B(x)}}{\int \text{d}x e^{-\beta U_A(x)}} = \frac{\int \text{d}x e^{-\beta U_A(x)} e^{\beta U_A(x)} e^{-\beta U_B(x)}}{\int \text{d}x e^{-\beta U_A(x)}} = \frac{\int \text{d}x e^{-\beta U_A(x)} e^{-\beta \Delta U_{AB}(x)}}{\int \text{d}x e^{-\beta U_A(x)}} = \Braket{e^{-\beta \Delta U_{AB}}}_A
$$
where $\Delta U_{AB} = U_B-U_A$.

The free energy can therefore be computed as
$$
\Delta F_{AB} = k_BT\log \Braket{e^{-\beta \Delta U_{AB}}}_A
$$

From an operational point of view, this means to perform a simulation in the system subject to the interaction $U_A$ and compute the standard average of the obseervable defined by the exponential of the energy difference $\Delta U_{AB}$.

Even if easy at first sight, this can be challenging if the two systems are very different from each other since when the energy difference of a configuration in the two states is large, the contribution arising from the exponential is very low.

To check if the system are close enough to allow for the method to work, a good way is to check the convergence of the mean of the zwanzig factor along the simulation. To this end, running averages (RA, see the function below) are a useful tool.

In [ ]:
def running_average(x, step, avg):

    if step == 0:
        avg = step
    else:
        avg = (avg*(step-1) + x)/step

    return avg 

As a first test case, let us try to switch directly from the source to the target system of the thermodynamic integration example. Also try running the simulation different times. 
 - Are the result for the free energy difference compatible with the thermodynamic integration? 
 - Are different replicas of the simulation giving the same result?
 - What about the convergence of the Zwanzig factor?

In [ ]:
equilibration_steps = 10000
total_steps = 10000000
output_frequency = 10

beta = 1
timestep = 0.001
diffusion_coefficient = 1

initial_x = np.zeros(2)

assert equilibration_steps < n_steps_per_simulation, "Make sure you don't equilibrate longer than you simulate."
assert output_frequency < n_steps_per_simulation, "Make sure you don't output less often than you simulate."
assert output_frequency > 0, "The output frequency needs to be larger than 0"

In [ ]:
previous_x = initial_x.copy()

total_frames = int(np.ceil((total_steps - equilibration_steps) / output_frequency))
trajectory = np.zeros((total_frames, 2))
zwanzig_factor_avg = np.zeros((total_frames+1))

for step in range(total_steps):

    current_force = ...
    
    previous_x = update_positions(previous_x, current_force, beta, timestep, diffusion_coefficient)

    if step >= equilibration_steps and step % output_frequency == 0:
        index = (step - equilibration_steps) // output_frequency
        trajectory[index] = previous_x
        zwanzig_factor = ...
        zwanzig_factor_avg[index+1] = running_average(zwanzig_factor, index, zwanzig_factor_avg[index])

delta_F_fep_direct = ...
print(f"∆F = {delta_F_fep_direct}")

Plot the running average of the zwanzig factor. It is possible to obtain a reference value for the Zwanzig factor by inverting the formula for $\Delta F_{AB}$ and using the result form the Thermodynamics integration.

$$
\Delta F_{AB} = k_BT\log \Braket{e^{-\beta \Delta U_{AB}}}_A \quad \Longrightarrow \quad \Braket{e^{-\beta \Delta U_{AB}}}_A = e^{\beta \Delta F_{AB}}
$$

In [ ]:
zwanzig_factor_from_TI = ...

In [ ]:
fig, ax = plt.subplots(1, figsize=(5,5), dpi=180)

plt.xlabel("frame")
plt.ylabel(r"$<\exp(-\beta \Delta U_{AB})>_A$")
plt.plot(zwanzig_factor_avg, label="Running Average (RA)")
plt.axhline(zwanzig_factor_avg[-1], ls = ":", label="Average", c="k")
plt.axhline(zwanzig_factor_from_TI, ls="-.", label="Reference", c="r")
plt.legend(frameon=False)
plt.show()

If the previous tests failed, a step-by-step approach can be pursued. 
To make sure that the energies between the two systems do not differ too much, proceed by intermediate steps, running different simulations (note that they can be run in parallel ideally) for intermediate states between the source and target systems and computing the free energy difference between each successive pair. 

To slowly switch between source and target space, the same linear protocol of the thermodynamics integration example can be used, or you can come up with different protocols. Think about what you need to do: For each intermediate step a source and a target system need to be defined and the simulation needs to run following the dynamics corresponding to the defined source system. 

- Is the zwanzig factor now converging for each simulation? 
- What information can you obtain from the free-energy difference between each pair of systems? 
- How to obtain the free energy difference between source and target space?
- Is the result for the free energy now stable with respect to running the simulation many times?

In [ ]:
l_simulations = 10
lambdas = np.linspace(0, 1, l_simulations)

equilibration_steps = 10000
n_steps_per_simulation = 1000000
output_frequency = 10

beta = 1
timestep = 0.001
diffusion_coefficient = 1

initial_x = np.zeros(2)

assert equilibration_steps < n_steps_per_simulation, "Make sure you don't equilibrate longer than you simulate."
assert output_frequency < n_steps_per_simulation, "Make sure you don't output less often than you simulate."
assert output_frequency > 0, "The output frequency needs to be larger than 0"

In [ ]:
delta_F_simulations = []
zwanzig_factor_avg_simulations = []

for il, l in enumerate(lambdas[:-1]):

    previous_x = initial_x.copy()

    total_frames = int(np.ceil((n_steps_per_simulation - equilibration_steps) / output_frequency))
    trajectory = np.zeros((total_frames, 2))
    zwanzig_factor_avg = np.zeros((total_frames+1))

    for step in range(n_steps_per_simulation):

        current_force = ...
        
        previous_x = update_positions(previous_x, current_force, beta, timestep, diffusion_coefficient)

        if step >= equilibration_steps and step % output_frequency == 0:
            index = (step - equilibration_steps) // output_frequency
            trajectory[index] = previous_x
            zwanzig_factor = ...
            zwanzig_factor_avg[index+1] = running_average(zwanzig_factor, index, zwanzig_factor_avg[index])

    zwanzig_factor_avg_simulations.append(zwanzig_factor_avg)

    delta_F_step = ...
    print(f"∆F = {delta_F_step}")
    delta_F_simulations.append(delta_F_step)

In [ ]:
fig, ax = plt.subplots(1, figsize=(5,5), dpi=180)

plt.xlabel("frame")
plt.ylabel(r"$<\exp(-\beta \Delta U_{i,i+1})>_A$")
plt.ylim(0,2.5)

for sim in range(l_simulations)[:-1]:
    plt.plot(zwanzig_factor_avg_simulations[sim], label=f"RA: {sim}, {sim+1}")
    plt.axhline(zwanzig_factor_avg_simulations[sim][-1], ls=":", c="k")

plt.legend(frameon=False, ncols=3)
plt.show()

In [ ]:
delta_F_fep_step = ...
print(f"∆F = {delta_F_fep_step}")

In [ ]:
fig, ax = plt.subplots(1, figsize=(5,5), dpi=180)

plt.xlabel("simulation")
plt.ylabel(r"$\Delta F_{i,i+1}$")
plt.plot(delta_F_simulations)
plt.show()